In [1]:
mkdir -p data/amazon

In [3]:
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf

--2021-03-27 10:49:52--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 167.114.174.41
Connecting to s2.q4cdn.com (s2.q4cdn.com)|167.114.174.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 869995 (850K) [application/pdf]
Saving to: ‘./data/amazon/Amazon-Q4-2020-Earnings-Release.pdf’

Amazon-Q4-2020-Earn 100%[===================>] 849.60K   895KB/s    in 0.9s    

2021-03-27 10:49:53 (895 KB/s) - ‘./data/amazon/Amazon-Q4-2020-Earnings-Release.pdf’ saved [869995/869995]

--2021-03-27 10:49:53--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 167.114.174.41
Connecting to s2.q4cdn.com (s2.q4cdn.com)|167.114.174.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 945573 (923K) [application/pdf]
Saving to: ‘./data/amazon/AMZN-Q3-2020-Earnings-Release.pdf’

AMZN-Q3

In [4]:
ls data/amazon

Amazon-Q4-2020-Earnings-Release.pdf  AMZN-Q3-2020-Earnings-Release.pdf
AMZN-Q1-2020-Earnings-Release.pdf    Q2-2020-Amazon-Earnings-Release.pdf


In [5]:
!nvidia-smi

Sat Mar 27 10:50:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-ksa8oa6f
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-ksa8oa6f
     |████████████████████████████████| 215kB 5.9MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 378kB 38.5MB/s 
     |████████████████████████████████| 327kB 50.5MB/s 
     |████████████████████████████████| 327kB 49.7MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 983kB 52.9MB/s 
     |████████████████████████████████| 5.5MB 45.5MB/s 
     |████████████████████████████████| 1.5MB 50.3MB/s 
     |████████████████████████████████| 143kB 57.8MB/s 
     |████████████████████████████████| 8.2MB 48.2MB/s 
     |████████████████████████████████| 225kB 39.5MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████

In [7]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

03/27/2021 10:56:49 - INFO - faiss.loader -   Loading faiss with AVX2 support.
03/27/2021 10:56:49 - INFO - faiss.loader -   Loading faiss.
03/27/2021 10:56:50 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [8]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [9]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

03/27/2021 10:59:06 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.077s]
03/27/2021 10:59:07 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.472s]
03/27/2021 10:59:07 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.188s]


In [17]:
directory = "data/amazon/"
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.preprocessor.preprocessor import PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["de","en"])

processor = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="word",
                         split_length=200,
                         split_respect_sentence_boundary=True)
docs = []
for filename in os.listdir(directory):

    # Run the conversion on each file (PDF -> 1x doc)
    d = converter.convert(os.path.join(directory, filename), meta=None)

    # clean and split each dict (1x doc -> multiple docs)
    d = processor.process(d)
    docs.extend(d)

# Let's have a look at the first 3 entries:
print(docs[:3])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
03/27/2021 11:18:27 - WARNING - haystack.preprocessor.preproce

[{'text': 'AMAZON.COM ANNOUNCES THIRD QUARTER RESULTS\nSEATTLE--(BUSINESS WIRE) October 29, 2020--Amazon.com, Inc. (NASDAQ: AMZN) today announced financial\nresults for its third quarter ended September 30, 2020. Operating cash flow increased 56% to $55.3 billion for the trailing twelve months, compared with $35.3 billion for\nthe trailing twelve months ended September 30, 2019. Free cash flow increased to $29.5 billion for the trailing twelve months, compared with $23.5 billion for the trailing\ntwelve months ended September 30, 2019. Free cash flow less principal repayments of finance leases and financing obligations increased to $18.4 billion for\nthe trailing twelve months, compared with $14.6 billion for the trailing twelve months ended September 30, 2019. Free cash flow less equipment finance leases and principal repayments of all other finance leases and financing\nobligations increased to $17.9 billion for the trailing twelve months, compared with $10.5 billion for the trailing

In [19]:
document_store.write_documents(docs)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

03/27/2021 11:18:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.352s]
03/27/2021 11:18:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.031s]


In [20]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [21]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

03/27/2021 11:19:28 - INFO - farm.utils -   Using device: CUDA 
03/27/2021 11:19:28 - INFO - farm.utils -   Number of GPUs: 1
03/27/2021 11:19:28 - INFO - farm.utils -   Distributed Training: False
03/27/2021 11:19:28 - INFO - farm.utils -   Automatic Mixed Precision: None
03/27/2021 11:19:29 - INFO - filelock -   Lock 139755531136976 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


03/27/2021 11:19:29 - INFO - filelock -   Lock 139755531136976 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


03/27/2021 11:19:31 - INFO - filelock -   Lock 139755861472784 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


03/27/2021 11:20:13 - INFO - filelock -   Lock 139755861472784 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
03/27/2021 11:20:25 - INFO - filelock -   Lock 139755853174672 acquired on /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


...

03/27/2021 11:20:31 - INFO - filelock -   Lock 139755513207184 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


03/27/2021 11:20:32 - INFO - filelock -   Lock 139755513207184 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


03/27/2021 11:20:32 - INFO - filelock -   Lock 139755513350992 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


03/27/2021 11:20:33 - INFO - filelock -   Lock 139755513350992 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
03/27/2021 11:20:33 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/27/2021 11:20:33 - INFO - farm.utils -   Using device: CUDA 
03/27/2021 11:20:33 - INFO - farm.utils -   Number of GPUs: 1
03/27/2021 11:20:33 - INFO - farm.utils -   Distributed Training: False
03/27/2021 11:20:33 - INFO - farm.utils -   Automatic Mixed Precision: None


03/27/2021 11:20:33 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
03/27/2021 11:20:33 - INFO - farm.infer -    0    0 
03/27/2021 11:20:33 - INFO - farm.infer -   /w\  /w\
03/27/2021 11:20:33 - INFO - farm.infer -   /'\  / \
03/27/2021 11:20:33 - INFO - farm.infer -     


In [22]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [53]:
prediction = pipe.run(query="Who is the new CEO?", top_k_retriever=20, top_k_reader=5)

03/27/2021 11:39:58 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.023s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 44.15 Batches/s]


In [55]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Andy Jassy',
        'context': 'ition to the role of Executive Chair in the third quarter '
                   'of 2021 and\n'
                   'Andy Jassy will become Chief Executive Officer at that '
                   'time. "Amazon is what it '},
    {   'answer': 'Andy Jassy',
        'context': 'ition to the role of Executive Chair in the third quarter '
                   'of 2021 and\n'
                   'Andy Jassy will become Chief Executive Officer at that '
                   'time. "Amazon is what it '},
    {   'answer': 'Andy Jassy',
        'context': 'ition to the role of Executive Chair in the third quarter '
                   'of 2021 and\n'
                   'Andy Jassy will become Chief Executive Officer at that '
                   'time. "Amazon is what it '},
    {   'answer': 'Andy Jassy',
        'context': ' and CEO Jeff Bezos will transition to role of Executive '
                   'Chair in Q3,\n'
                   'Andy Jassy to 

In [14]:
!apt-get install xpdf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cups-bsd cups-client cups-common gsfonts gsfonts-x11 libcupsfilters1
  libcupsimage2 libmotif-common libxm4 poppler-data poppler-utils
  xfonts-encodings xfonts-utils
Suggested packages:
  cups inetutils-inetd | inet-superserver update-inetd xpp smbclient
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  cups-bsd cups-client cups-common gsfonts gsfonts-x11 libcupsfilters1
  libcupsimage2 libmotif-common libxm4 poppler-data poppler-utils
  xfonts-encodings xfonts-utils xpdf
0 upgraded, 14 newly installed, 0 to remove and 30 not upgraded.
Need to get 7,044 kB of archives.
After this operation, 26.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/un